# Loads the needed packages and reads the EMG data

In [61]:
import pandas as pd
import numpy as np
from read_emg import *
import seaborn as sns
from IPython.display import Image

plt.style.use('default')
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['legend.title_fontsize'] = 14
plt.rcParams['axes.labelsize'] = 16
pd.options.display.float_format = "{:,.3f}".format

In [40]:
### Reads the EMG data
df = pd.read_pickle('DataCollection.pkl')
df['Trial_num'] = df['Trial_num'].astype(int)

# Function that calculates the cross-correlation (Similarity)

In [41]:
Image(url= "Similarity.png", width=600, height=600)

In [53]:
### Calculating the similarity using the channel means
def cross_correlation_vector(A, B):
    A_vector = []
    B_vector = []
    for channel in range(0, 8):
        A_temp = A[:, channel] 
        A_channel_mean = A_temp.mean()
        A_vector.append(A_channel_mean)
        
        B_temp = B[:, channel]
        B_channel_mean = B_temp.mean()
        B_vector.append(B_channel_mean)
        
    A_vector = np.array(A_vector)
    B_vector = np.array(B_vector)
    C = np.multiply(A_vector, B_vector).sum() / (np.sqrt((A_vector**2).sum() * (B_vector**2).sum()))

    return C

# Function that flip the channels around ( 1 2 3 4 to 4 3 2 1)

In [44]:
def flipping_sensor(df_unflipped):
    A = df_unflipped.iloc[:, :8]
    A = A.loc[:, ::-1].values
    
    df_flipped = df_unflipped.copy()
    df_flipped.iloc[:, :8] = A
    
    return df_flipped

# Function that calculates the similarity between non-shifted and shifted channels

In [62]:
def CC_to_shift(df, subject_reference, subject_to_rotate, calibration_gesture, session_reference, session_to_rotate, plot = False):
    shift_list = []
    
    if plot == True:
        fig, ax = plt.subplots(3, 4, figsize = (21, 12))
        R = 0 
        Col = 0
        
    print(f'Current Subject: {subject_to_rotate}')
    df_A = df[(df['ID'] == subject_reference) & 
              (df['Gesture'] == calibration_gesture) & 
              (df['Trial_num'] == 3) & 
              (df['session'] == session_reference)].iloc[:, :8]

    A_value = df_A.iloc[100:, :8].rolling(20).mean().dropna().values

    shift_range = range(0, 8)

    max_CC = 0
    final_status = False
    final_shift = 0
    for flipped in [False, True]:
        if flipped == False:    
            df_B = df[(df['ID'] == subject_to_rotate) & 
                      (df['Gesture'] == calibration_gesture) & 
                      (df['Trial_num'] <= 3) & 
                      (df['session'] == session_to_rotate)].iloc[:, :8]
        else:
            print('Flipping the sensor\n')
            df_B = flipping_sensor(df[(df['ID'] == subject_to_rotate)  & 
                                      (df['Gesture'] == calibration_gesture) & 
                                      (df['Trial_num'] <= 3) & 
                                      (df['session'] == session_to_rotate)]).iloc[:, :8]
        

        B = df_B.iloc[100:, :8].rolling(20).mean().dropna()     
         
        CC_list = []
        for shift in shift_range:
            print(f'Shift: {shift}')

            B_value = pd.concat((B,B), axis = 1).iloc[:,  0 + shift:8 + shift].values

            C = cross_correlation_vector(A_value, B_value)
            print(f'CC: {C:.4f}\n')
            CC_list.append(C)
        try:
            shift_idx = np.where(CC_list == np.max(CC_list))[0][0]
            shift_list.append(shift_idx)

            if np.max(CC_list) > max_CC:
                max_CC = np.max(CC_list)
                final_status = flipped
                final_shift = shift_idx
                
        except Exception as e:
            print(e)

    print(f'\nFlipping needed? {final_status}\n')

    return final_status, shift_list


In [63]:
def shift_to_rotation(df, shift, subject_reference, subject_to_rotate, session_reference, session_to_rotate, flipped):
    df_train = df[(df['ID'] == subject_reference) & (df['session'] == session_reference) & (df['Trial_num'] <=3)]
        
    df_test = df[(df['ID'] == subject_to_rotate) & (df['session'] == session_to_rotate) & (df['Trial_num'] <=3)]
    

    df_test_temp = df_test.iloc[:, :8].copy()
    
    if flipped == True:
        df_test_temp = flipping_sensor(df_test.iloc[:, :8])
        df_test = flipping_sensor(df_test)
        
    df_test_calibrated = pd.concat((pd.concat((df_test_temp, df_test_temp), axis = 1).iloc[:,  0 + shift:8 + shift], df_test.iloc[:, 8:]), axis = 1)
    df_test_calibrated = pd.DataFrame(df_test_calibrated.values, columns = df_test.columns)
            
    return df_train, df_test, df_test_calibrated

# Calculates the similarity of gestures between 2 sessions in the same subject


In [64]:
subject_list = list(df['ID'].unique())
gesture_list = ['2', '4', '5', '6', '7', '8', '9']
label_list = ['Cylinder Grasp', 'Wrist Extension', 'Fist', 'Finger mass extension', 'Opposition', 'Wrist / Finger extension', 'Lateral Pinch']
int_to_label = dict(zip(gesture_list, label_list))
stats = list()

for subject in ['0001', '1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1111', '1234', '9999'][:]:
    print('\n###############################')
    print(f'Current subject: {subject}')
    print('###############################\n')

        
    flipped, shift_list = CC_to_shift(df, 
                        subject_reference = subject, 
                        subject_to_rotate = subject, 
                        calibration_gesture = '8', 
                        session_reference = 'S1', 
                        session_to_rotate = 'S2')
    flipped = False
    
    if flipped == False:
        shift = shift_list[0]
    else:
        shift = shift_list[1]    
        
    
    df_reference, df_original, df_calibrated = shift_to_rotation(df, shift, 
                                                                 subject_reference = subject, 
                                                                 subject_to_rotate = subject,
                                                                 session_reference = 'S1', 
                                                                 session_to_rotate = 'S2', 
                                                                 flipped = flipped)
    print('Gesutre: Cross-correlation')
    for gesture in gesture_list[:]:
        df_reference_to_plot = df_reference[(df_reference['Gesture'] == gesture)]
        df_rotated_to_plot = df_calibrated[(df_calibrated['Gesture'] == gesture)]
        
        A = df_reference_to_plot.iloc[:, :8].values
        B = df_rotated_to_plot.iloc[:, :8].values

        C= cross_correlation_vector(A, B)
        print(f'{int_to_label[gesture]}: {C:.2f}')
        stats.append({'ID':subject,
                      'shift': shift,
                      'Gesture': int_to_label[gesture],
                      'CC': C})


###############################
Current subject: 0001
###############################

Current Subject: 0001
Shift: 0
CC: 0.9717

Shift: 1
CC: 0.7386

Shift: 2
CC: 0.5124

Shift: 3
CC: 0.4231

Shift: 4
CC: 0.4457

Shift: 5
CC: 0.4936

Shift: 6
CC: 0.6393

Shift: 7
CC: 0.9050

Flipping the sensor

Shift: 0
CC: 0.8091

Shift: 1
CC: 0.9691

Shift: 2
CC: 0.8278

Shift: 3
CC: 0.6491

Shift: 4
CC: 0.5565

Shift: 5
CC: 0.4279

Shift: 6
CC: 0.3669

Shift: 7
CC: 0.5230


Flipping needed? False

Gesutre: Cross-correlation
Cylinder Grasp: 0.95
Wrist Extension: 0.99
Fist: 0.95
Finger mass extension: 0.98
Opposition: 0.96
Wrist / Finger extension: 0.97
Lateral Pinch: 0.97

###############################
Current subject: 1001
###############################

Current Subject: 1001
Shift: 0
CC: 0.9893

Shift: 1
CC: 0.8728

Shift: 2
CC: 0.7161

Shift: 3
CC: 0.5958

Shift: 4
CC: 0.5747

Shift: 5
CC: 0.6698

Shift: 6
CC: 0.8345

Shift: 7
CC: 0.9622

Flipping the sensor

Shift: 0
CC: 0.8383

Shift: 1
CC

# Printing the similarity of gestures between 2 sessions in the same subject

In [67]:
stats = pd.DataFrame(stats)
stats.groupby('Gesture')['CC'].mean()

Gesture
Cylinder Grasp             0.970
Finger mass extension      0.980
Fist                       0.975
Lateral Pinch              0.960
Opposition                 0.965
Wrist / Finger extension   0.987
Wrist Extension            0.980
Name: CC, dtype: float64